In [ ]:
import cPickle
import matplotlib.pyplot as plt
import numpy as np
import os
from mpl_toolkits.basemap import Basemap
from IPython import display
from time import sleep
from scipy.stats import pearsonr
os.chdir("/Users/zbutler/research/fire_prediction/")
from util.daymonth import increment_day, day2monthday
from geometry.grid_conversion import ak_bb
%matplotlib inline

In [ ]:
with open("data/precipitation/tp_ak.pkl") as fpkl:
    tp_ak = cPickle.load(fpkl)
print "Loaded %d length dict" % len(tp_ak)

In [ ]:
ex = tp_ak[(2010, 12, 1, '0600')]
ex

In [ ]:
hours = ['0000', '0600', '1200', '1800']
day = 1
month = 1
year = 2010
overall_mean_arr = []
overall_std_arr = []
hours_mean_arr = dict()
hours_std_arr = dict()
for hour in hours:
    hours_mean_arr[hour] = []
    hours_std_arr[hour] = []
tp_min = np.inf
tp_max = -np.inf
    
while year < 2014:
    overall_mat = np.zeros(ex.shape)
    for hour in hours:
        time_key = (year, month, day, hour)
        if time_key not in tp_ak:
            hours_mean_arr[hour].append(0)
            hours_std_arr[hour].append(0)
        else:
            mat = tp_ak[time_key]
            hours_mean_arr[hour].append(np.mean(mat))
            hours_std_arr[hour].append(np.std(mat))
            overall_mat += mat
            mn = np.min(mat)
            mx = np.max(mat)
            if mn < tp_min:
                tp_min = mn
            if mx > tp_max:
                tp_max = mx
    overall_mean_arr.append(np.mean(overall_mat))
    overall_std_arr.append(np.std(overall_mat))
    year, month, day = increment_day(year, month, day)
    
overall_mean_arr = np.array(overall_mean_arr)
overall_std_arr = np.array(overall_std_arr)
for hour in hours:
    hours_mean_arr[hour] = np.array(hours_mean_arr[hour])
    hours_std_arr[hour] = np.array(hours_std_arr[hour])    
    
plt.plot(overall_mean_arr, 'b-')
plt.plot(overall_mean_arr + overall_std_arr, 'r--')
plt.plot(overall_mean_arr - overall_std_arr, 'r--')
plt.ylabel('Mean cumulative daily rainfall in Alaska')
plt.xlabel('Time (2010-2013)')
plt.show()

fig,axarrs = plt.subplots(nrows=2, ncols=2, figsize=(14, 12))
axarr = [None, None, None, None]
axarr[0] = axarrs[0][0]
axarr[1] = axarrs[0][1]
axarr[2] = axarrs[1][0]
axarr[3] = axarrs[1][1]
for i,hour in enumerate(hours):
    axarr[i].plot(hours_mean_arr[hour], 'b-')
    axarr[i].plot(hours_mean_arr[hour] + hours_std_arr[hour], 'r--')
    axarr[i].plot(hours_mean_arr[hour] - hours_std_arr[hour], 'r--')
    axarr[i].set_ylabel('Mean cumulative rainfall in Alaska (%s)' % hour)
    axarr[i].set_xlabel('Time (2010-2013)')
plt.show()

In [ ]:
day2monthday(220)

In [ ]:
bb = ak_bb
day = 15
month = 7
year = 2013
while month < 9:
    for hour in hours:
        display.clear_output(wait=True)
        plt.figure(figsize=(10,10))
        mp = Basemap(projection="merc",
                     lat_0=bb[0], lon_0=bb[2],
                     llcrnrlat=bb[0],
                     llcrnrlon=bb[2],
                     urcrnrlat=bb[1],
                     urcrnrlon=bb[3])
        mp.drawcoastlines()
        mp.imshow(tp_ak[(year,month,day,hour)], vmin=tp_min, vmax=tp_max)
        mp.colorbar()
        plt.title("Cumulative rainfal %d/%d/%d, %s" % (month, day, year, hour))
        plt.show()
    year, month, day = increment_day(year, month, day)


In [ ]:
def aggregate_rainfall(tp_dict, min_year, max_year, shape):
    hours = ['0000','0600','1200','1800']
    ret_dict = dict()
    year = min_year
    month = 1
    day = 1
    min_val = np.inf
    max_val = -np.inf
    while year <= max_year:
        n_valid = 0
        mat = np.zeros(shape)
        for hour in hours:
            key = (year, month, day, hour)
            if key in tp_dict:
                n_valid += 1
                mat += tp_dict[key]
        if n_valid:
            agg_key = (year, month, day)
            ret_dict[agg_key] = (len(hours) / float(n_valid)) * mat
            mn = np.min(ret_dict[agg_key])
            mx = np.max(ret_dict[agg_key])
            if mn < min_val:
                min_val = mn
            if mx > max_val:
                max_val = mx
        year, month, day = increment_day(year,month,day)
    ret_dict['min'] = min_val
    ret_dict['max'] = max_val
    return ret_dict

tp_agg_dict = aggregate_rainfall(tp_ak, 2010, 2013, ex.shape)
print "Created aggregate dict of shape " + str(len(tp_agg_dict))

In [ ]:
# Now, bring it all together with fires
with open("data/ak_fires.pkl") as fpkl:
    ak_fires = cPickle.load(fpkl)
print "loaded %d fires" % len(ak_fires)

In [ ]:
bb = ak_bb
day = 1
month = 7
year = 2013
ex = tp_agg_dict[(2010, 1, 1)]
shape = ex.shape

prev_lats = []
prev_longs = []
while month < 9:
    todays_fires = ak_fires[(ak_fires.year == year) & (ak_fires.month == month) &
                            (ak_fires.day == day)]
    display.clear_output(wait=True)
    plt.figure(figsize=(10,10))
    mp = Basemap(projection="merc",
                 lat_0=bb[0], lon_0=bb[2],
                 llcrnrlat=bb[0],
                 llcrnrlon=bb[2],
                 urcrnrlat=bb[1],
                 urcrnrlon=bb[3])
    mp.drawcoastlines()
    if (year,month,day) in tp_agg_dict:
        mp.imshow(tp_agg_dict[(year,month,day)], vmin=tp_agg_dict['min'], vmax=tp_agg_dict['max'])
    else:
        mp.imshow(np.zeros(shape), vmin=tp_agg_dict['min'], vmax=tp_agg_dict['max'])
    mp.colorbar()
    plt.title("Cumulative rainfall %d/%d/%d" % (month, day, year))
    if len(prev_lats):
        mp.plot(prev_longs, prev_lats, 'ko')
    if len(todays_fires):
        today_longs, today_lats = mp(np.array(todays_fires.long), np.array(todays_fires.lat))
        mp.plot(today_longs, today_lats, 'ro')
        prev_lats += list(today_lats)
        prev_longs += list(today_longs)
    plt.show()
    sleep(.4)
    year, month, day = increment_day(year, month, day)

In [ ]:
# Huh that seemed weird. Let's look at a time series of average precipitation vs temperature
fires = []
rain = []

month = 5
day = 1
year = 2010
while month < 10:
    fires.append(len(ak_fires[(ak_fires.year == year) & (ak_fires.month == month) &
                            (ak_fires.day == day)]))
    if (year, month, day) in tp_agg_dict:
        rain.append(np.mean(tp_agg_dict[(year,month,day)]))
    else:
        rain.append(0)
    year,month,day = increment_day(year,month,day)
        
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12,10))
ax1 = plt.subplot(211)
plt.plot(fires, 'r--')
plt.title('Number of detections')

ax2 = plt.subplot(212, sharex=ax1)
plt.plot(rain, 'b--')
plt.title('Mean cumulative rainfall')
fig.tight_layout()
plt.show()

In [ ]:
# correlations
for year in xrange(2010, 2014):
    fires = []
    rain = []

    month = 5
    day = 1
    while month < 10:
        fires.append(len(ak_fires[(ak_fires.year == year) & (ak_fires.month == month) &
                                (ak_fires.day == day)]))
        if (year, month, day) in tp_agg_dict:
            rain.append(np.mean(tp_agg_dict[(year,month,day)]))
        else:
            rain.append(0)
        year,month,day = increment_day(year,month,day)
        
    print "correlation in year %d: %s" % (year, str(pearsonr(fires, rain)))